<a href="https://colab.research.google.com/github/Dhyanesh-Panchal/GSR-research-and-stress-detection/blob/master/DNN_model_with_only_phasicGSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

---
### Prepare the the Data

In [11]:
data_sub1 = pd.read_csv("https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/preprocessed_data/sub_1.csv")
data_sub1.head()

,time,gsr,video,valence,arousal,EDA_Clean,EDA_Phasic,EDA_Tonic
0,0,0.398182,10,5.0,5.0,0.398273,-0.074354,0.410863
1,50,0.395989,10,5.0,5.0,0.397157,-0.087782,0.410951
2,100,0.396867,10,5.0,5.0,0.396104,-0.100518,0.411042
3,150,0.394235,10,5.0,5.0,0.395134,-0.112338,0.411133
4,200,0.394673,10,5.0,5.0,0.394251,-0.123223,0.411227


In [2]:
# windowing function
def rolling_window(df,window_size):
    windows = []
    for i in range(len(df)-window_size+1):
        single_window = df.iloc[i:i+window_size]
        # print(single_window)
        windows.append(single_window)
    return windows

In [10]:
def pre_process(df):
  windowed_data = rolling_window(df,100)
  intermediate_array = []
  for frame in windowed_data:
    intermediate_array.append(list(frame["EDA_Phasic"]))

  phasic_windowed = pd.DataFrame(np.array(intermediate_array))
  phasic_windowed.shape

  X = np.array(phasic_windowed).reshape(len(phasic_windowed),100,1)
  # inserted the initial arousal value during the "start" of window
  X = np.concatenate((X,np.array(data_sub1['arousal'][99:]).reshape(len(df['arousal']) - 99,1,1)) , axis=1)

  # it is the final valence-arousal value "after" the window
  y = np.array(df[['valence','arousal']])[99:]

  return X,y

In [12]:
X,y = pre_process(data_sub1)

In [13]:
X.shape

(48933, 101, 1)

In [14]:
y.shape

(48933, 2)

In [18]:
# Create a CNN model.
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(64, 10, activation='relu', input_shape=(101, 1)),
  tf.keras.layers.MaxPooling1D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1,activation='linear')
])

# Compile the CNN model.
model.compile(loss='mse', optimizer='adam')

In [ ]:
fitting = model.fit(X, y[:,1], epochs=32)  #Just test

--> Achieved the minimum loss so far: MSE Loss:- 1.1337

---
#### Heavy Cell, Applying to batch
Training on rest of 25 subjects, 5 subjects left for testing:

In [17]:
fitting_array = []

In [ ]:
for i in range(1,26):
  print(f"\n\n********* *********  Iteration - {i} *********  *********\n\n")
  data = pd.read_csv(f"https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/preprocessed_data/sub_{i}.csv")
  X,y = pre_process(data)
  fitting_array.append(model.fit(X, y[:,1], epochs=32 , validation_split = 0.2))



********* *********  Iteration - 1 *********  *********


Epoch 1/32
1224/1224 [==============================] - 6s 4ms/step - loss: 0.3640 - val_loss: 1.7527e-05
Epoch 2/32
1224/1224 [==============================] - 6s 5ms/step - loss: 1.8657e-05 - val_loss: 5.2398e-06
Epoch 3/32
1224/1224 [==============================] - 5s 4ms/step - loss: 6.6990e-06 - val_loss: 4.9288e-06
Epoch 4/32
1224/1224 [==============================] - 5s 4ms/step - loss: 1.2033e-04 - val_loss: 6.2786e-05
Epoch 5/32
1224/1224 [==============================] - 6s 5ms/step - loss: 4.4315e-04 - val_loss: 3.8671e-06
Epoch 6/32
1224/1224 [==============================] - 5s 4ms/step - loss: 1.3983e-04 - val_loss: 7.1156e-06
Epoch 7/32
1224/1224 [==============================] - 5s 4ms/step - loss: 1.8739e-04 - val_loss: 2.9945e-04
Epoch 8/32
1224/1224 [==============================] - 5s 4ms/step - loss: 1.2302e-04 - val_loss: 1.5091e-05
Epoch 9/32
1224/1224 [==============================] - 5s 4ms/s

In [ ]:
model.save('./final_model.h5',save_format='h5')

In [ ]:
from google.colab import files

In [ ]:
files.download('./final_model.h5')

In [ ]:
fitting.history